In [ ]:
import pandas as pd
import numpy as np
import math

from datetime import datetime as date
from pandas._libs.tslibs.timestamps import Timestamp
import calendar

from dateutil import parser

In [ ]:
#@title Name In File Here
inFileName = "Walking Birding Replication 2 2024" #@param {type:"string"}
##Name What the in file is called

In [ ]:
#@title Name Out File Here
outFileName = "TestReplicatedEvents" #@param {type:"string"}
##Name What you want the out file to be called

In [ ]:
#@title How many rows per export?
rowLimit = 5000 #@param {type:"number"}
rowLimit = rowLimit -5
##Name The export will be split into appropriate files

In [ ]:
#@title Start Date
startDateBound = "1/1/2024" #@param {type:"string"}
##Name Starting date to export

In [ ]:
#@title End Date
endDateBound = "12/31/2024" #@param {type:"string"}
##Name Ending date to export

In [ ]:

messageLog = []
auditTrailLog = []
errorLog = []

#This adds the excell extension if it is not detected in the FileNames
fileType = 'xlsx'

if fileType == "xlsx":
    try:
        if '.xlxs' not in inFileName:
            inFileName = inFileName + '.xlsx'

        #this takes the '.xlxs' if its in the out file beacuse it's added in later

        if '.xlxs' in outFileName:
            outFileName = outFileName.replace('.xlsx','')

        #create DataFrame
        inDf = pd.read_excel(inFileName)

    except Exception as e:
        errorLog.append('error generating dataframe with file: ')
        errorLog.append(inFileName)
        errorLog.append(e)
        print('error generating dataframe with file: ' + inFileName)
        print(e)


In [ ]:
salesForceColumns = ["Event Name", "Location Name", "Street", "City", "State", "Zip/Postal Code", "Start Date", "End Date", "Start Time", "End Time", "Time Zone", "Description", "Event Category", "Parking / Access Instructions", "Event Leader Contact: Full Name", "Event Leader Email", "Event Leader Phone", "Paid Event", "Location Type", "External Identifier", "Event Link", "Event Documents", "Event Documents2", "Account Name", "Account ID", "Event ID", "External Account ID", "E3HID", "E3H Club ID", "Passion", "Event Replication", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday", "Monthly Frequency"]

platformColumns = ["Event Name", "Venue Name", "Event Street Address", "Event Street Address 2", "Event City", "Event State", "Event ZIP", "Venue County", "Event Start Date", "Event End Date", "Event Start Time", "Event End Time", "Time Zone", "Event Description", "Event Type", "Important Details", "Event Leader First Name", "Event Leader Last Name", "Event Leader Email", "Event Leader Phone", "Paid Event", "Virtual Event", "Virtual Medium", "Virtual Url", "Virtual Url Passcode", "Virtual Dial-In Number", "Virtual Dial-In Passcode", "External Id", "File #1", "File #1 Title", "File #2", "File #2 Title", "File #3", "File #3 Title", "File #4", "File #4 Title", "File #5", "File #5 Title", "File #6", "File #6 Title", "File #7", "File #7 Title", "File #8", "File #8 Title", "Club Name", "Event Replication", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday", "Monthly Frequency"]

In [ ]:
#this splits the date up into year, month, date so it can be interpreted as a 'DateTime' object
#param is the date as seen in the excel file as a string in the format of M/D/YYYY or MM/DD/YYYY
#returns calDate as a string in the format of YYYY/MM/DD

def formatDate(d):
    try:
        if str(type(d)) == "<class 'str'>":
            y = d[d.rindex('/')+1:]
            m = d[:d.index('/')]
            d = d[d.index('/')+1:d.rindex('/')]

            if len(m) == 1:
                m = '0' + m

            if len(d) == 1:
                d = '0' + d

            calDate = y + '-' + m + '-' + d

            if len(calDate) < 9:
              calDate = '20' + calDate

        elif str(type(d)) == "<class 'pandas._libs.tslibs.timestamps.Timestamp'>" or "<class 'datetime.datetime'>":

            calDate = str(d).split()[0]

        else:

            calDate = '2021-01-01'
            print(d, type(d))

    except Exception as e:
        errorLog.append('error with formatDate(): ')
        errorLog.append(str(d))
        errorLog.append(e)
        print('error with formatDate(): ' + str(d))
        print(e)

    return calDate

In [ ]:
#sees what type of format was uploaded

isPlatform = False
isSalesForce = False
originalLength = 0

if (inDf.columns.tolist() == salesForceColumns):

    isSalesForce = True

elif (inDf.columns.tolist() == platformColumns):

    isPlatform = True
    #The Platform format has two redundant rows. This drops them
    exampleRows = inDf.loc[[0,1]]
    inDf = inDf.drop(inDf.index[[0,1]])
    inDf = inDf.reset_index()
else:
    print("format not recognized")
    errorLog.append("format not recognized\n")

originalLength = len(inDf.index)

if isPlatform:
    auditTrailLog.append("Platform Format detected\n")
if isSalesForce:
    auditTrailLog.append("SalesForce Format detected\n")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#This finds the unique frequencies in the Replication column. Good for finding typos

uniquieFrequencies = []

for frequency in inDf['Event Replication']:
    if frequency not in uniquieFrequencies:
        uniquieFrequencies.append(frequency)

In [ ]:
#Loops through the Start Dates and prints date rejections. These can be handled case by case

badDateFormats = []

if isSalesForce:

    for count, (dateStart, dateEnd) in enumerate(zip(inDf['Start Date'], inDf['End Date'])):

        try:
          d = formatDate(dateStart)
          calStartDate = date.fromisoformat(d)
          inDf['Start Date'][count] = calStartDate

        except:
          badDateFormats.append("Row number: " + str(count+1) + ", Date in cell" + dateStart)

        try:
          d = formatDate(dateEnd)
          calEndDate = date.fromisoformat(d)
          inDf['End Date'][count] = calEndDate

        except:
          badDateFormats.append("Row number: " + str(count+1) + ", Date in cell" + dateEnd)

elif isPlatform:

    for count, (dateStart, dateEnd) in enumerate(zip(inDf['Event Start Date'], inDf['Event End Date'])):

        try:
          d = formatDate(dateStart)
          calStartDate = date.fromisoformat(d)
          inDf['Event Start Date'][count] = calStartDate

        except:
          badDateFormats.append("Row number: " + str(count+1) + ", Date in cell" + dateStart)

        try:
          d = formatDate(dateEnd)
          calEndDate = date.fromisoformat(d)
          inDf['Event End Date'][count] = calEndDate

        except:
          badDateFormats.append("Row number: " + str(count+1) + ", Date in cell " + dateEnd)


if(len(badDateFormats) > 0):
    errorLog.append('Error(s) with imported dates')
    errorLog.append(badDateFormats)

<ipython-input-13-402d745fadaa>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inDf['Start Date'][count] = calStartDate
<ipython-input-13-402d745fadaa>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inDf['End Date'][count] = calEndDate
<ipython-input-13-402d745fadaa>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inDf['Start Date'][count] = calStartDate
<ipython-input-13-402d745fadaa>:20: SettingWithCopyWarning: 
A value is tr

In [ ]:
#converts the appropriate start and end dates into DateTimes

try:
    startDateBound = formatDate(startDateBound)
    startDateBound = date.fromisoformat(startDateBound)

    endDateBound = formatDate(endDateBound)
    endDateBound = date.fromisoformat(endDateBound)

    assert (date.fromisoformat(formatDate(startDateBound)).year == date.fromisoformat(formatDate(endDateBound)).year)

except Exception as e:
    print('Invalid Start and/or end date '+ e)
    errorLog.append('Invalid Start and/or end date '+ e)


In [ ]:
#This looks at the Replication Column and returns a list with all the days that are found
#param - replication cell
#returns - list with the days found

def getWeekdayNumber(daysCell):

    days = []

    if ',' in daysCell:
        days = daysCell.split(',')

    elif '/' in daysCell:
        days = daysCell.split('/')

    elif ',' not in daysCell and '/' not in daysCell:
        days.append(daysCell)


    returnedDays = []

    for day in days:

        day= day.lower().strip()

        if  'Monday'.lower() == day or 'M'.lower() == day or 'Mon'.lower() == day:
            returnedDays.append(0)
        elif  'Tuesday'.lower() == day or 'Tu'.lower() == day or 'Tue'.lower() == day  or 'Tues'.lower() == day:
            returnedDays.append(1)
        elif  'Wednesday'.lower() == day or 'W'.lower() == day or 'Wen'.lower() == day or 'Wens'.lower() == day :
            returnedDays.append(2)
        elif  'Thursday'.lower() == day or 'Th'.lower() == day or 'Thu'.lower() == day or 'Thur'.lower() == day  or 'Thurs'.lower() == day:
            returnedDays.append(3)
        elif  'Friday'.lower() == day or 'F'.lower() == day or 'Fri'.lower() == day:
            returnedDays.append(4)
        elif  'Saturday'.lower() == day or 'Sa'.lower() == day or 'Sat'.lower() == day:
            returnedDays.append(5)
        elif  'Sunday'.lower() == day or 'Su'.lower() == day or 'Sun'.lower() == day:
            returnedDays.append(6)

        else:
          errorLog.append('day not recognized: ')
          errorLog.append(day)
          print('day not recognized: ' + day)

    return returnedDays

In [ ]:
#This looks at the replication cell and returns the 'monthly frequency'
#in this case, it will assume it is repeated every week of the month

#params - replication cell
#reuturns - 0-4 list that corresponds to a 'monthly frequency'
#0 means every week

def getMonthlyFrequency(freq):
    freq = str(freq)
    if freq != 'nan':
        if '0' in freq:
            freq = freq[0]
        try:
            if ',' in freq:
              return (freq.replace(' ', '').replace('  ', '').split(','))
            elif len(freq) == 1:
                return [str(int(freq))]
            else:
                return ['0']
        except:
            errorLog.append('Error with Montly Frequency: ')
            errorLog.append(freq)
            return ['0']

    else:
        return ['0']
        errorLog.append('Error with Montly Frequency: ')
        errorLog.append(freq)


In [ ]:
#this function is not needed, but useful for debugging

#params year, month and day
#returns day of the week

def get_day(year, month, day):
    d = date.datetime(year, month, day)
    offset = 1-d.weekday() #weekday = 1 means tuesday
    if offset < 0:
        offset+=7
    return d+date.timedelta(offset)


In [ ]:
#this function compares the start and end day and returns all the months in between
#params are the start and end date (in the DateTime format)
#returns a list of the months

def getMonthsRequested(startD, endD):

    startMonth = startD.month
    endMonth = endD.month

    if startD.year == endD.year:
        yearsRequested = startD.year
        return list(range(startMonth, endMonth+1)), startD.year
    else:
        print('not ready for multi year use')
        errorLog.append('not ready for multi year use')
        #this does not accomidate ranges that span over two calendar years

In [ ]:
#this function considers the 'monthly frequency' ie) every first, 'weekly frequency' ie) Tuesday, start date and end date
#it returns a list with all of the appropriate dats

def getFutureEventDates(monthlyFreq, weekFreq, startD, endD, startTime, endTime, inDf, currentIndex):

    monthsReqested, yearsRequested = getMonthsRequested(startD, endD)

    replicatedDates = []

    replicatedStartTimes = []

    replicatedEndTimes = []

    calYear = yearsRequested

    for month in monthsReqested:

        daysInMonth = calendar.monthrange(calYear, month)[1]

        firstDayOfMonth = date(calYear, month, 1)

        firstWeekdayOfMonth = date.weekday(firstDayOfMonth)

        weekNumber = 1

        for dayze in range(daysInMonth):

            trialDate = date(calYear, month, dayze+1)

            trialWeekDay = date.weekday(trialDate)

            formattedDate = str(trialDate.month) + '/' + str(trialDate.day) + '/' + str(trialDate.year)


            if isSalesForce:

                if (trialDate > date.fromisoformat(str(inDf['Start Date'][currentIndex])) and trialDate < date.fromisoformat(str(inDf['End Date'][currentIndex]))):

                      if (trialWeekDay in weekFreq and '0' in monthlyFreq):
                          replicatedDates.append(formattedDate)
                          returnedStartTime, returnedEndTime = getEventStartAndEndTimes(trialWeekDay, startTime, endTime, inDf, currentIndex)
                          replicatedStartTimes.append(returnedStartTime)
                          replicatedEndTimes.append(returnedEndTime)

                      elif (trialWeekDay in weekFreq and '1' in monthlyFreq and weekNumber == 1):

                          replicatedDates.append(formattedDate)
                          returnedStartTime, returnedEndTime = getEventStartAndEndTimes(trialWeekDay, startTime, endTime, inDf, currentIndex)
                          replicatedStartTimes.append(returnedStartTime)
                          replicatedEndTimes.append(returnedEndTime)

                      elif (trialWeekDay in weekFreq and '2' in monthlyFreq and weekNumber == 2):
                          replicatedDates.append(formattedDate)
                          returnedStartTime, returnedEndTime = getEventStartAndEndTimes(trialWeekDay, startTime, endTime, inDf, currentIndex)
                          replicatedStartTimes.append(returnedStartTime)
                          replicatedEndTimes.append(returnedEndTime)

                      elif (trialWeekDay in weekFreq and '3' in monthlyFreq and weekNumber == 3):
                          replicatedDates.append(formattedDate)
                          returnedStartTime, returnedEndTime = getEventStartAndEndTimes(trialWeekDay, startTime, endTime, inDf, currentIndex)
                          replicatedStartTimes.append(returnedStartTime)
                          replicatedEndTimes.append(returnedEndTime)

                      elif (trialWeekDay in weekFreq and '4' in monthlyFreq and weekNumber == 4):
                          replicatedDates.append(formattedDate)
                          returnedStartTime, returnedEndTime = getEventStartAndEndTimes(trialWeekDay, startTime, endTime, inDf, currentIndex)
                          replicatedStartTimes.append(returnedStartTime)
                          replicatedEndTimes.append(returnedEndTime)

            if isPlatform:

                  if (trialDate > date.fromisoformat(str(inDf['Event Start Date'][currentIndex])) and trialDate < date.fromisoformat(str(inDf['Event End Date'][currentIndex]))):

                      if (trialWeekDay in weekFreq and '0' in monthlyFreq):
                          replicatedDates.append(formattedDate)
                          returnedStartTime, returnedEndTime = getEventStartAndEndTimes(trialWeekDay, startTime, endTime, inDf, currentIndex)
                          replicatedStartTimes.append(returnedStartTime)
                          replicatedEndTimes.append(returnedEndTime)

                      elif (trialWeekDay in weekFreq and '1' in monthlyFreq and weekNumber == 1):
                          replicatedDates.append(formattedDate)
                          returnedStartTime, returnedEndTime = getEventStartAndEndTimes(trialWeekDay, startTime, endTime, inDf, currentIndex)
                          replicatedStartTimes.append(returnedStartTime)
                          replicatedEndTimes.append(returnedEndTime)

                      elif (trialWeekDay in weekFreq and '2' in monthlyFreq and weekNumber == 2):
                          replicatedDates.append(formattedDate)
                          returnedStartTime, returnedEndTime = getEventStartAndEndTimes(trialWeekDay, startTime, endTime, inDf, currentIndex)
                          replicatedStartTimes.append(returnedStartTime)
                          replicatedEndTimes.append(returnedEndTime)

                      elif (trialWeekDay in weekFreq and '3' in monthlyFreq and weekNumber == 3):
                          replicatedDates.append(formattedDate)
                          returnedStartTime, returnedEndTime = getEventStartAndEndTimes(trialWeekDay, startTime, endTime, inDf, currentIndex)
                          replicatedStartTimes.append(returnedStartTime)
                          replicatedEndTimes.append(returnedEndTime)

                      elif (trialWeekDay in weekFreq and '4' in monthlyFreq and weekNumber == 4):
                          replicatedDates.append(formattedDate)
                          returnedStartTime, returnedEndTime = getEventStartAndEndTimes(trialWeekDay, startTime, endTime, inDf, currentIndex)
                          replicatedStartTimes.append(returnedStartTime)
                          replicatedEndTimes.append(returnedEndTime)

            if (trialDate.day) % 7 == 0:
                weekNumber += 1

    return replicatedDates, replicatedStartTimes, replicatedEndTimes


In [ ]:
def getStandardEventLength(startTime, endTime):
    return (parser.parse(endTime) - parser.parse(startTime))

In [ ]:
def convert24(str1):

    if str1[0] != '0':
        str1 = '0' + str1

    if str1[-2:] == "AM" and str1[:2] == "12":
        return "00" + str1[2:-2]

    # remove the AM
    elif str1[-2:] == "AM":
        return str1[:-2]

    # Checking if last two elements of time
    # is PM and first two elements are 12
    elif str1[-2:] == "PM" and str1[:2] == "12":
        return str1[:-2]

    else:

        # add 12 to hours and remove PM
        if (len(str1)) > 8:
            return str(int(str1[:2]) + 12) + str1[2:8]
        else:
            return str(int(str1[:2]) + 12) + str1[2:5]

In [ ]:
#this finds the new start and end times that are needed for the event replicaiton
#it is based on if there's a time in the Weekday column and the duration that is in the original event row

def getEventStartAndEndTimes(weeklyFrequencyNumber, startTime, endTime, inDf, currentIndex):
    try:
        returnedStartTime = ''
        returnedEndTime = ''

        weekDayName = ''
        weekDayTime = ''

        if weeklyFrequencyNumber == 0: weekDayName = 'Monday'
        if weeklyFrequencyNumber == 1: weekDayName = 'Tuesday'
        if weeklyFrequencyNumber == 2: weekDayName = 'Wednesday'
        if weeklyFrequencyNumber == 3: weekDayName = 'Thursday'
        if weeklyFrequencyNumber == 4: weekDayName = 'Friday'
        if weeklyFrequencyNumber == 5: weekDayName = 'Saturday'
        if weeklyFrequencyNumber == 6: weekDayName = 'Sunday'


        if len(str(inDf[weekDayName][currentIndex])) > 1 and str(inDf[weekDayName][currentIndex]) != 'nan':

            weekDayTime = str(inDf[weekDayName][currentIndex])

            if weekDayTime[-1].lower() == 'm':
                weekDayTime = convert24(weekDayTime)

            if len(str(weekDayTime)) > 12:
                sampleDay = date.fromisoformat('2022-01-01T' + weekDayTime.strptime('%H:%M'))

            sampleDay = date.fromisoformat('2022-01-01T' + weekDayTime)

            returnedStartTime = inDf[weekDayName][currentIndex]
            eventNewEnd = sampleDay + getStandardEventLength(str(startTime), str(endTime))
            returnedEndTime = eventNewEnd.time()

        elif len(str(inDf[weekDayName][currentIndex])) == 0 or str(inDf[weekDayName][currentIndex]) == 'nan':

            returnedStartTime = startTime
            returnedEndTime = endTime

    except Exception as e:
        errorLog.append("getStartAndEndTimes error at ")
        errorLog.append(currentIndex)
        errorLog.append(str(e))
        print('exception', e)


    return returnedStartTime, returnedEndTime

In [ ]:
#This section uses the above functions to add the new dates needed
  #into a new column ['New n Dates'] as a list to corresponding row

newDates = []

inDf['New n Dates'] = inDf['Event Replication']
inDf['New n Start Times'] = inDf['Event Replication']
inDf['New n End Times'] = inDf['Event Replication']

for count, freq in enumerate(inDf['Event Replication']):
    #print(freq)
    #print(getWeekdayNumber(freq))
    try:
        if isPlatform:
            #print(inDf['Monthly Frequency'][count])

            inDf['New n Dates'][count], inDf['New n Start Times'][count], inDf['New n End Times'][count] = getFutureEventDates(getMonthlyFrequency(inDf['Monthly Frequency'][count]), getWeekdayNumber(freq), date.fromisoformat(formatDate(startDateBound)), date.fromisoformat(formatDate(endDateBound)), inDf['Event Start Time'][count], inDf['Event End Time'][count], inDf, count)

        elif isSalesForce:
            inDf['New n Dates'][count], inDf['New n Start Times'][count], inDf['New n End Times'][count] = getFutureEventDates(getMonthlyFrequency(inDf['Monthly Frequency'][count]), getWeekdayNumber(freq), date.fromisoformat(formatDate(startDateBound)), date.fromisoformat(formatDate(endDateBound)), inDf['Start Time'][count], inDf['End Time'][count], inDf, count)

    except Exception as e:
        errorLog.append('Error Generating new dates for event: ')
        errorLog.append(inDf['Event Name'][count])
        errorLog.append(str(e))
        print(e)

exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T

<ipython-input-23-f7d52a1fe47f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inDf['New n Dates'][count], inDf['New n Start Times'][count], inDf['New n End Times'][count] = getFutureEventDates(getMonthlyFrequency(inDf['Monthly Frequency'][count]), getWeekdayNumber(freq), date.fromisoformat(formatDate(startDateBound)), date.fromisoformat(formatDate(endDateBound)), inDf['Start Time'][count], inDf['End Time'][count], inDf, count)
<ipython-input-23-f7d52a1fe47f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inDf['New n Dates'][count], inDf['New n Start Times'][count], inDf['New n End Times'][count] = getFutureEvent


exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01

<ipython-input-23-f7d52a1fe47f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inDf['New n Dates'][count], inDf['New n Start Times'][count], inDf['New n End Times'][count] = getFutureEventDates(getMonthlyFrequency(inDf['Monthly Frequency'][count]), getWeekdayNumber(freq), date.fromisoformat(formatDate(startDateBound)), date.fromisoformat(formatDate(endDateBound)), inDf['Start Time'][count], inDf['End Time'][count], inDf, count)
<ipython-input-23-f7d52a1fe47f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inDf['New n Dates'][count], inDf['New n Start Times'][count], inDf['New n End Times'][count] = getFutureEvent


exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01

...

<ipython-input-23-f7d52a1fe47f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inDf['New n Dates'][count], inDf['New n Start Times'][count], inDf['New n End Times'][count] = getFutureEventDates(getMonthlyFrequency(inDf['Monthly Frequency'][count]), getWeekdayNumber(freq), date.fromisoformat(formatDate(startDateBound)), date.fromisoformat(formatDate(endDateBound)), inDf['Start Time'][count], inDf['End Time'][count], inDf, count)
<ipython-input-23-f7d52a1fe47f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inDf['New n Dates'][count], inDf['New n Start Times'][count], inDf['New n End Times'][count] = getFutureEvent

exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T

<ipython-input-23-f7d52a1fe47f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inDf['New n Dates'][count], inDf['New n Start Times'][count], inDf['New n End Times'][count] = getFutureEventDates(getMonthlyFrequency(inDf['Monthly Frequency'][count]), getWeekdayNumber(freq), date.fromisoformat(formatDate(startDateBound)), date.fromisoformat(formatDate(endDateBound)), inDf['Start Time'][count], inDf['End Time'][count], inDf, count)
<ipython-input-23-f7d52a1fe47f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inDf['New n Dates'][count], inDf['New n Start Times'][count], inDf['New n End Times'][count] = getFutureEvent

exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T09:00 '
exception Invalid isoformat string: '2022-01-01T

<ipython-input-23-f7d52a1fe47f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inDf['New n Dates'][count], inDf['New n Start Times'][count], inDf['New n End Times'][count] = getFutureEventDates(getMonthlyFrequency(inDf['Monthly Frequency'][count]), getWeekdayNumber(freq), date.fromisoformat(formatDate(startDateBound)), date.fromisoformat(formatDate(endDateBound)), inDf['Start Time'][count], inDf['End Time'][count], inDf, count)
<ipython-input-23-f7d52a1fe47f>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inDf['New n Dates'][count], inDf['New n Start Times'][count], inDf['New n End Times'][count] = getFutureEvent

In [ ]:
#this loops through the new dates that are needed and adds each one as a new row with the new date
#it also adds the new start and end time.
#If a unquie start time is specified on the day column, it makes an end time based on the duration of the original row
#Uses the getEventStartAndEndTimes()

for count, (name, newDatesList, newStartTimesList, newEndTimesList) in enumerate(zip(inDf['Event Name'], inDf['New n Dates'], inDf['New n Start Times'], inDf['New n End Times'])):

    auditTrailLog.append(name + 'copied to the following dates ...')

    for (newDate, newStartTime, newEndTime) in zip(newDatesList, newStartTimesList, newEndTimesList):

        try:
            if isSalesForce:
                inDf = inDf.append(inDf.iloc[count])
                inDf.iloc[-1, inDf.columns.get_loc('Start Date')] = str(newDate)
                log = '       ' + str(newDate)

                if newStartTime.strftime("%I:%M %p")[0] == '0':
                    inDf.iloc[-1, inDf.columns.get_loc('Start Time')] = newStartTime.strftime("%I:%M %p")[1:]
                    log = log + ', at: ' + str(newStartTime.strftime("%I:%M %p")[1:])
                else:
                    inDf.iloc[-1, inDf.columns.get_loc('Start Time')] = newStartTime.strftime("%I:%M %p")
                    log = log + ', at: ' +  str(newStartTime.strftime("%I:%M %p"))

                if newEndTime.strftime("%I:%M %p")[0] == '0':
                    inDf.iloc[-1, inDf.columns.get_loc('End Time')] = newEndTime.strftime("%I:%M %p")[1:]
                    log = log + ' - '+ str(newEndTime.strftime("%I:%M %p")[1:])
                else:
                    inDf.iloc[-1, inDf.columns.get_loc('End Time')] = newEndTime.strftime("%I:%M %p")
                    log = log + ' - '+ str(newEndTime.strftime("%I:%M %p"))


            if isPlatform:

                inDf = inDf.append(inDf.iloc[count])
                inDf.iloc[-1, inDf.columns.get_loc('Event Start Date')] = str(newDate)
                log = '       ' + str(newDate)

                if newStartTime.strftime("%I:%M %p")[0] == '0':
                    inDf.iloc[-1, inDf.columns.get_loc('Event Start Time')] = newStartTime.strftime("%I:%M %p")[1:]
                    log = log + ', at : ' + str(newStartTime.strftime("%I:%M %p")[1:])
                else:
                    inDf.iloc[-1, inDf.columns.get_loc('Event Start Time')] = newStartTime.strftime("%I:%M %p")
                    log = log + ', at : ' +  str(newStartTime.strftime("%I:%M %p"))

                if newEndTime.strftime("%I:%M %p")[0] == '0':
                    inDf.iloc[-1, inDf.columns.get_loc('Event End Time')] = newEndTime.strftime("%I:%M %p")[1:]
                    log = log + ' - '+ str(newEndTime.strftime("%I:%M %p")[1:])
                else:
                    inDf.iloc[-1, inDf.columns.get_loc('Event End Time')] = newEndTime.strftime("%I:%M %p")
                    log = log + ' - '+ str(newEndTime.strftime("%I:%M %p"))

            auditTrailLog.append(log)

        except Exception as e:
            errorLog.append('Error Populating event: ')
            errorLog.append(name)
            errorLog.append('     at start time: ')
            errorLog.append(newStartTime)
            errorLog.append('and end time: ')
            errorLog.append(newEndTime)
            errorLog.append(str(e))

    auditTrailLog.append('\n\n\n')


In [ ]:
#drops unnecessary columns and rows

try:
    inDf = inDf.drop(['New n Start Times', 'New n End Times', "Event Replication", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday", "Monthly Frequency"], 1)
    if isPlatform:
        inDf = inDf[originalLength:]
        exampleRows = exampleRows.drop(["Event Replication", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday", "Monthly Frequency"], 1)
        inDf = pd.concat([exampleRows, inDf], axis=0)

    if isSalesForce:
        inDf = inDf[originalLength:]
except Exception as e:
    errorLog.append('error dropping cols/rows: ')
    errorLog.append(e)

In [ ]:
try:
    #This outputs the split Dfs into excel files with indecies
    splits = math.ceil(len(inDf) / rowLimit)
    outDfs = []
    for i in range(splits):
        outDfs.append(inDf.iloc[rowLimit*(i):rowLimit*(i+1),:])

  #This splits the df into a list in with row limits
    for count, splitDf in enumerate (outDfs):
        if fileType == 'xlsx':
            outFileNameAndIndex = outFileName + '_' + str(count+1) + '.xlsx'
            splitDf.to_excel(outFileNameAndIndex, index = False)

except Exception as e:
    errorLog.append('Error generating export files:')
    errorLog.append(e)

In [ ]:
#Writes exportLog file
try:
    with open('exportLog.txt', 'w') as exportFile:
        if len(errorLog) > 0:
            exportFile.write('See errors:\n')
            for element in errorLog:
                exportFile.write('%s\n' % element)
            exportFile.write(':\n')

        for element in auditTrailLog:
            exportFile.write('%s\n' % element)

except Exception as e:
    errorLog.append('Error generating export log:')
    errorLog.append(e)

In [ ]:
try:
    #the new rows are inserted and this column is no longer needed
    inDf = inDf.drop('New n Dates', 1)

    #this just duplicates the (new) start date to each end date
    if isSalesForce:
        inDf['End Date'] = inDf['Start Date']

    if isPlatform:
        inDf['Event End Date'] = inDf['Event Start Date']
        inDf = inDf.drop(['index'], 1)

except Exception as e:
    errorLog.append('error sorting columns ', e)

TypeError: list.append() takes exactly one argument (2 given)

In [ ]:
#Errors are shown below
print(errorLog)